In [1]:
import warnings
warnings.filterwarnings('ignore')

import shutil
import pytorch_lightning as pl
import torch
import yaml
import os
import pickle
import logging
from audio_classification.tools import *
import optuna
from pytorch_lightning import loggers as pl_loggers
from optuna.integration import PyTorchLightningPruningCallback

In [2]:
import yaml
with open("/nfs/students/winter-term-2020/project-1/project-1/audio_classification/configs/m11_bmw.yaml", "r") as config_file:
    cfg = yaml.load(config_file)

In [3]:
from pytorch_lightning import Callback

class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""
    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [4]:
def save_model(model, file_name, directory = "models"):
    model = model.cpu()
    model_dict = {cfg["MODEL"]["NAME"]:{"state_dict":model.state_dict(), "hparams": model.hparams}}
    if not os.path.exists(directory):
        os.makedirs(directory)
    pickle.dump(model_dict, open(os.path.join(directory, file_name), 'wb', 4))

In [5]:
def objective(trial):
    # Use this callback to collect the validation accuracies
        
    metrics_callback = MetricsCallback()
    
    trial_hparams = {"batch_size": 6, 
                     "learning_rate": trial.suggest_loguniform("learning_rate", 1e-7, 1e-1),
                     "weight_decay": trial.suggest_loguniform("weight_decay", 1e-8, 1e-3)
                    }

    train_loader, val_loader, test_loader, class_weights = get_dataloader(cfg, trial_hparams, transform=get_transform(cfg))
    
    if class_weights is not None:
        class_weights = torch.tensor(class_weights).to(device=torch.device('cuda'))
    
    early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_acc")
    trainer = pl.Trainer(gpus=cfg["SOLVER"]["NUM_GPUS"],
                         min_epochs=cfg["SOLVER"]["MIN_EPOCH"],
                         max_epochs=30,
                         progress_bar_refresh_rate=10,
                         callbacks=[metrics_callback, early_stop_callback],
                         logger=True,
                        )
    
    model = get_model(cfg, class_weights, trial_hparams, train_loader, val_loader)
    
    model.prepare_data()
    trainer.fit(model)

    save_model(model, '{}.p'.format(trial.number), "checkpoints")

    # return validation accuracy from latest model, as that's what we want to maximize by our hyper param search
    return metrics_callback.metrics[-1]["val_acc"]

In [6]:
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=50, timeout=86000)

[I 2020-11-26 23:24:26,393] A new study created in memory with name: no-name-92e8870c-b845-4f22-ae08-28cae74d338e
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


[I 2020-11-26 23:39:34,024] Trial 0 finished with value: 0.6666666865348816 and parameters: {'learning_rate': 0.014425843243837163, 'weight_decay': 2.427559370693672e-07}. Best is trial 0 with value: 0.6666666865348816.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


[I 2020-11-26 23:54:39,865] Trial 1 finished with value: 0.3499999940395355 and parameters: {'learning_rate': 0.0012356654508578959, 'weight_decay': 0.0003248537151877524}. Best is trial 0 with value: 0.6666666865348816.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


[I 2020-11-27 00:09:52,081] Trial 2 finished with value: 0.7666666507720947 and parameters: {'learning_rate': 0.0004049346269594982, 'weight_decay': 1.4351249927646852e-07}. Best is trial 2 with value: 0.7666666507720947.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


[I 2020-11-27 00:25:13,062] Trial 3 finished with value: 0.6666666865348816 and parameters: {'learning_rate': 3.5008134126663386e-07, 'weight_decay': 1.2455710288031881e-06}. Best is trial 2 with value: 0.7666666507720947.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


...

[I 2020-11-27 11:00:35,893] Trial 45 finished with value: 0.6833333373069763 and parameters: {'learning_rate': 0.0003126805901398549, 'weight_decay': 3.3478246832489854e-07}. Best is trial 17 with value: 0.800000011920929.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


[I 2020-11-27 11:15:49,416] Trial 46 finished with value: 0.6666666865348816 and parameters: {'learning_rate': 4.656618717898221e-05, 'weight_decay': 1.5693550252981776e-06}. Best is trial 17 with value: 0.800000011920929.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


[I 2020-11-27 11:31:11,234] Trial 47 finished with value: 0.4166666567325592 and parameters: {'learning_rate': 0.0001267264414788311, 'weight_decay': 1.3422896767154703e-07}. Best is trial 17 with value: 0.800000011920929.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


[I 2020-11-27 11:46:33,065] Trial 48 finished with value: 0.6666666865348816 and parameters: {'learning_rate': 0.0006164582610803727, 'weight_decay': 8.006811541115696e-08}. Best is trial 17 with value: 0.800000011920929.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   


[I 2020-11-27 12:01:59,521] Trial 49 finished with value: 0.4000000059604645 and parameters: {'learning_rate': 7.434767020918564e-06, 'weight_decay': 9.095392887429728e-07}. Best is trial 17 with value: 0.800000011920929.


In [7]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
best_trial = study.best_trial

print("  Value: {}".format(best_trial.value))

print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 50
Best trial:
  Value: 0.800000011920929
  Params: 
    learning_rate: 1.5984826253924876e-05
    weight_decay: 1.2743200395718975e-05


In [8]:
shutil.rmtree("checkpoints")